## Collaborative Filtering

Integrantes
* Santiago
* Jose
* Patricia
* Yedam Fortiz 119523

### Carga de paqueteria

In [1]:
import pandas as pd
import numpy as np
import numpy.ma as ma
import random as r
from sklearn.model_selection import train_test_split
from sklearn.metrics import ndcg_score

In [2]:
r.seed(1234)

### Carga de Informacion

In [3]:
links = pd.read_csv("links_small.csv")
ratings = pd.read_csv("ratings_small.csv")
ratings = ratings.drop("timestamp",axis=1)

### Por ahora consideraremos únicamente 1500 lineas de ratings, no los 100,000 registros ya que el algoritmo no está optimizado para una base de datos tan grande

In [4]:
X_train,X_test = train_test_split(ratings.head(1500),test_size=0.3,random_state=1234)

In [5]:
ratings_wide = X_train.pivot_table(index = 'userId',
                                   columns = 'movieId',
                                   values = 'rating')

### Creacion de matrices

In [6]:
links_values = np.array(links)
links_features = np.array(links.columns)
ratings_values = np.array(ratings_wide)
ratings_features = np.array(ratings_wide.columns)

### Funcion de costo

$$ \large J(X)  =  \frac{1}{2} \sum_{(a,i \in D)} (Y_{ai} - [UV^T]_{ai} )^2 + \frac{\lambda}{2} \sum_{a=1}^n\sum_{j=1}^k U_{aj}^2 + \frac{\lambda}{2} \sum_{i=1}^m\sum_{j=1}^k V_{ij}^2  $$

* Seleccionamos $V$ al azar y la dejamos fija y optimizamos con respecto a $U$
* Una vez actualizada la $U$, la dejamos fija y optimizamos con respecto a $V$
* Repetimos hasta que converja (variaciones entre las estimaciones de los vectores es pequeña) (optimo local)

#### Minimizacion Alternada

In [7]:
def gradiente_U (Y,U,V,k,lambda_):
    
    if k!=1:
        U_V = U.dot(V.T)
    else:
        U_V = np.outer(U,V)
    
    na = np.isnan(Y)
    
    #Tratamiento especial NA
    #gradiente = -dot_na(Y-U_V, V) + lambda_*U
    gradiente = -(Y-U_V)@V + lambda_*U
    
    return gradiente

In [8]:
def gradiente_V (Y,U,V,k,lambda_):
    
    if k!=1:
        U_V = U.dot(V.T)
    else:
        U_V = np.outer(U,V)
    
    na = np.isnan(Y)
    
    #Tratamiento especial NA
    #gradiente = -dot_na((Y-U_V).T, U) + lambda_*V
    gradiente = -(Y-U_V).T@U + lambda_*V
    
    return gradiente

In [9]:
def dot_na(X,Y):
    
    n,m = X.shape
    lista = []
    
    for i in range(n):
        pos_na = ~np.isnan(X[i,:])
        lista.append(X[i,pos_na]@Y[pos_na])
        lista_ = np.array(lista)
    
    return lista_ 

In [10]:
def descenso_gradiente_U(Y,U,V,k,lambda_,eta,epsilon):
    
    U_ = U
    
    while True:
        U_aux = U_
        gradiente_u = gradiente_U(Y,U_,V,k,lambda_)
        U_ = U_ - eta * gradiente_u
        
        if (np.linalg.norm(U_aux-U_))< epsilon:
            break

    return U_

In [11]:
def descenso_gradiente_V(Y,U,V,k,lambda_,eta,epsilon):
    
    V_ = V
    
    while True:
        V_aux = V_
        gradiente_v = gradiente_V(Y,U,V_,k,lambda_)
        V_ = V_ - eta * gradiente_v
        
        if (np.linalg.norm(V_aux-V_))< epsilon:
            break

    return V_

In [12]:
def minimizacion_alternada(Y,k,lambda_,eta,epsilon):
    """
    Objetivo:
    Realizar minimizacion alternada
    
    Insumo:
    Y - Matriz a evaluar
    k - Hiperparametro para obtimizar funcion de costo
    lambda - Hiperparametro de regularizacion
    eta - Tamaño de paso
    epsilon - Criterio de paro
    
    Resultado:
    U - Sentimiento general de cada usuario hacia las peliculas
    V - Como cada una de las peliculas es percibidas por los usuarios
    
    """
    
    n,m = Y.shape
    
    Y = np.nan_to_num(Y)
    
    U = np.random.uniform(low = 0,high = (1/np.sqrt(k)),size = [n,k])
    V = np.random.uniform(low = 0,high = (1/np.sqrt(k)),size = [m,k])
    
    #Optimizar U
    U_final = descenso_gradiente_U(Y,U,V,k,lambda_,eta,epsilon)
    
    #Optimizar V
    V_final = descenso_gradiente_V(Y,U_final,V,k,lambda_,eta,epsilon)
    
    return U_final,V_final

Podemos hacer la revisión con esta matriz

In [13]:
Y = np.array([[5, np.nan, 7], 
              [1, 1, np.nan]])

In [14]:
Y

array([[ 5., nan,  7.],
       [ 1.,  1., nan]])

In [15]:
U_Y,V_Y = minimizacion_alternada(Y,k=2,lambda_=0.1,eta=0.01,epsilon=1e-3)

In [16]:
print(Y)
print(U_Y@V_Y.T)
print(np.linalg.norm(np.nan_to_num(Y)-U_Y@V_Y.T))

[[ 5. nan  7.]
 [ 1.  1. nan]]
[[ 4.99114292e+00 -1.49238422e-03  6.99200775e+00]
 [ 8.44586362e-01  9.28919502e-01 -9.37683960e-03]]
0.1715759669644307


### En estos momentos no se consideró el procedimiento de tratamiento especial de los Nulos, ya que el programa tarda mucho tiempo en correr. Por ahora estamos considerando los registros nulos como 0.

Haremos la estimación para la información de las películas

In [17]:
U_final,V_final = minimizacion_alternada(ratings_wide,k=10,lambda_=0.1,eta=0.01,epsilon=1e-3)

In [18]:
print(np.linalg.norm(np.nan_to_num(ratings_wide)-U_final@V_final.T))

47.91649305803306


## Validacion cruzada

Justificar la elección de k .\
Como hacer la validacion cruzada, como implementar el test:\
    *Ya estimamos el modelo bajo train, que se hace con el test

### Verificaremos el valor del ndcg score para diferentes valores de k

In [33]:
U_a,V_a = minimizacion_alternada(ratings_wide,k=5,lambda_=0.1,eta=0.01,epsilon=1e-3)
U_b,V_b = minimizacion_alternada(ratings_wide,k=10,lambda_=0.1,eta=0.01,epsilon=1e-3)
U_c,V_c = minimizacion_alternada(ratings_wide,k=15,lambda_=0.1,eta=0.01,epsilon=1e-3)
U_d,V_d = minimizacion_alternada(ratings_wide,k=50,lambda_=0.1,eta=0.01,epsilon=1e-3)

In [34]:
ndcga = ndcg_score(np.nan_to_num(ratings_wide), U_a@V_a.T)
ndcgb = ndcg_score(np.nan_to_num(ratings_wide), U_b@V_b.T)
ndcgc = ndcg_score(np.nan_to_num(ratings_wide), U_c@V_c.T)
ndcgd = ndcg_score(np.nan_to_num(ratings_wide), U_d@V_d.T)

In [35]:
pd.DataFrame({"k":[5,10,15,50],
             "NDCG":[ndcga,ndcgb,ndcgc,ndcgd]})
#[ndcg_score(np.nan_to_num(ratings_wide), U_5@V_5.T)]

,k,NDCG
0,5,0.718625
1,10,0.919949
2,15,0.996974
3,50,1.000000


### 5 Mejores recomendaciones

Como hacer las recomendaciones para las 5 peliculas por usuario

#### Pendientes

Agregar más teoría al documento